## Installing dependancies for Sentiment Analysis

In [2]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.2/553.2 kB 10.5 MB/s eta 0:00:00


In [3]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00


## Importing essential library files

In [4]:
import pandas as pd
import emoji
import re
import random

## Setup top 30 used emojis dict with their individual sentiments

In [5]:
emoji_sentiment_dict = {
    '😊': 'positive',  # Smiling face
    '😂': 'positive',  # Face with tears of joy
    '❤️': 'positive',  # Red heart
    '🥺': 'positive',  # Pleading face (can be neutral but often seen as positive)
    '😍': 'positive',  # Smiling face with heart-eyes
    '🤣': 'positive',  # Rolling on the floor laughing
    '🙏': 'positive',  # Folded hands (often gratitude or prayer)
    '😭': 'negative',  # Loudly crying face (can also be positive, but mostly negative)
    '💕': 'positive',  # Two hearts
    '😘': 'positive',  # Face blowing a kiss
    '😔': 'negative',  # Pensive face
    '🤔': 'neutral',   # Thinking face
    '😎': 'positive',  # Smiling face with sunglasses (cool)
    '😏': 'neutral',   # Smirking face
    '💪': 'positive',  # Flexed biceps (strength, determination)
    '👀': 'neutral',   # Eyes (often neutral or curious)
    '🎉': 'positive',  # Party popper
    '💔': 'negative',  # Broken heart
    '🙌': 'positive',  # Raising hands (celebration, success)
    '🥳': 'positive',  # Party face
    '😢': 'negative',  # Crying face
    '😇': 'positive',  # Smiling face with halo (angelic)
    '😜': 'positive',  # Winking face with tongue (playful)
    '😡': 'negative',  # Angry face
    '😋': 'positive',  # Face savoring food (happy, content)
    '😉': 'positive',  # Winking face
    '🥰': 'positive',  # Smiling face with hearts
    '😱': 'negative',  # Face screaming in fear
    '👏': 'positive'   # Clapping hands
}

## Generate Text with Emojis for Analysis

In [6]:
# Generate random text data
def generate_random_text(length=10):
    words = ["happy", "sad", "amazing", "bad", "great", "awesome", "love", "hate", "celebrate", "wonderful"]
    return ' '.join(random.choice(words) for _ in range(length))

# Create the DataFrame
data = []
num_samples = random.randint(250, 500)  # Random number of samples between 250 and 500

for _ in range(num_samples):
    text = generate_random_text()
    emojis = ''.join(random.choices(list(emoji_sentiment_dict.keys()), k=random.randint(1, 5)))
    combined_text = text + ' ' + emojis  # Combine text and emojis
    data.append({'combined_text': combined_text})

df = pd.DataFrame(data)
df.head()

,combined_text
0,amazing celebrate awesome great bad happy love...
1,amazing amazing sad awesome love awesome bad c...
2,sad amazing great wonderful great bad celebrat...
3,happy amazing hate sad happy great love celebr...
4,love wonderful love love happy hate great amaz...


## Sentiment Analysis

In [7]:
# Map emoji sentiment dynamically
def get_emoji_sentiment(emojis):
    sentiments = [emoji_sentiment_dict.get(e, 'neutral') for e in emojis]
    return 'positive' if sentiments.count('positive') > sentiments.count('negative') else 'negative'

In [8]:
# Extract emojis from combined text for sentiment analysis
df['emojis'] = df['combined_text'].apply(lambda x: ''.join(filter(lambda c: c in emoji_sentiment_dict, x)))

# Apply emoji sentiment analysis on the emojis extracted
df['emoji_sentiment'] = df['emojis'].apply(get_emoji_sentiment)

In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

# Function to get text sentiment using VADER
def get_text_sentiment(text):
    score = analyzer.polarity_scores(text)
    if score['compound'] >= 0.05:
        return 'positive'
    elif score['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

In [10]:
# Apply text sentiment analysis
df['text_sentiment'] = df['combined_text'].apply(get_text_sentiment)

# Combine text and emoji sentiment
def overall_sentiment(row):
    if row['text_sentiment'] == 'positive' or row['emoji_sentiment'] == 'positive':
        return 'positive'
    elif row['text_sentiment'] == 'negative' or row['emoji_sentiment'] == 'negative':
        return 'negative'
    else:
        return 'neutral'

df['overall_sentiment'] = df.apply(overall_sentiment, axis=1)

In [11]:
df.head()

,combined_text,emojis,emoji_sentiment,text_sentiment,overall_sentiment
0,amazing celebrate awesome great bad happy love...,😎🙏,positive,positive,positive
1,amazing amazing sad awesome love awesome bad c...,🙏😜😋,positive,positive,positive
2,sad amazing great wonderful great bad celebrat...,🤔😡🙌,negative,positive,positive
3,happy amazing hate sad happy great love celebr...,💪,positive,positive,positive
4,love wonderful love love happy hate great amaz...,😜😊😇,positive,positive,positive
